In [7]:
import sympy as sp
import numpy as np
import plotly.graph_objects as go
import kaleido as kld

C:\Users\maria\AppData\Roaming\Python\Python313\site-packages\kaleido\__init__.py:14: UserWarning: 


This means that static image generation (e.g. `fig.write_image()`) will not work.

Please upgrade Plotly to version 6.1.1 or greater, or downgrade Kaleido to version 0.2.1.

  from .kaleido import Kaleido


In [8]:
def newton_raphson(f, df, x0, tol=1e-6, max_iter=100):
    x = x0
    iterations = [x0]

    for i in range(max_iter):
        fx = f(x)
        dfx = df(x)

        if dfx == 0:
            raise ZeroDivisionError("Derivada zero. Método falhou.")

        x_new = x - fx / dfx

        if np.isnan(x_new) or np.isinf(x_new):
            raise ValueError("Divergência numérica.")

        iterations.append(x_new)

        if abs(x_new - x) < tol:
            return x_new, iterations

        x = x_new

    raise ValueError("Número máximo de iterações atingido.")



In [15]:


def plot_newton_plotly(f, raiz, iteracoes):
    """
    Plot the function and the iterations of the Newton-Raphson method using Plotly.

    Parameters:
    f : callable
        The function to plot.
    raiz : float
        The root found by the method.
    iteracoes : list
        List of x values where the iterations occurred.
    """
    x_vals = np.linspace(min(iteracoes) - 1, max(iteracoes) + 1, 4000)
    y_vals = f(x_vals)

    fig = go.Figure()

    if isinstance(f, sp.Basic):
        f_expr = sp.simplify(f)
    else:
        f_expr = f.__name__ if hasattr(f, '__name__') else str(f)

    # Plot the function
    fig.add_trace(go.Scatter(x=x_vals, y=y_vals, mode='lines', name='f(x)'))

    # Plot the x-axis and y-axis
    fig.add_trace(go.Scatter(x=[min(x_vals), max(x_vals)], y=[0, 0], mode='lines', line=dict(color='black', width=1), showlegend=False))
    fig.add_trace(go.Scatter(x=[0, 0], y=[min(y_vals), max(y_vals)], mode='lines', line=dict(color='black', width=1), showlegend=False))

    # Plot the iterations and tangents
    for i in range(len(iteracoes) - 1):
        xi = iteracoes[i]
        yi = f(xi)
        # Tangent line
        fig.add_trace(go.Scatter(x=[xi, iteracoes[i+1]], y=[yi, 0], mode='lines', line=dict(color='red', dash='dash'), showlegend=False))
        fig.add_trace(go.Scatter(x=[xi], y=[yi], mode='markers', marker=dict(color='red', size=8), showlegend=False))
        fig.add_trace(go.Scatter(x=[iteracoes[i+1]], y=[0], mode='markers', marker=dict(color='green', size=8), showlegend=False))

    # Plot the root
    fig.add_trace(go.Scatter(x=[raiz], y=[f(raiz)], mode='markers', marker=dict(color='blue', size=12, symbol='star'), name=f'Root: {raiz:.4f}'))

    fig.update_layout(
        title=f"Newton-Raphson Method: Root at x = {raiz:.4f} <br>Function: {f_expr}",
        xaxis_title="x",
        yaxis_title="f(x)",
        legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
        width=800,
        height=500
        )
    fig.show()

In [16]:
locals_dict = {
    "pi": sp.pi,
    "e": sp.E,
    "ln": sp.log,
    "log": sp.log,
    "sin": sp.sin,
    "cos": sp.cos,
    "tan": sp.tan,
    "exp": sp.exp,
    "sqrt": sp.sqrt,
}

In [17]:
x = sp.symbols('x')

f = input("Digite uma função de x (ex: sin(pi * x)): ")
f_expr = sp.sympify(f, locals=locals_dict)

f_prime = sp.diff(f_expr, x)
print(f"Função: {f_expr}, Derivada: {f_prime}")

Função: -1 + 3**(-sin(x)), Derivada: -log(3)*cos(x)/3**sin(x)


In [18]:
x = sp.symbols('x')
f_num = sp.lambdify(x, f, 'numpy')
f_prime_num = sp.lambdify(x, f_prime, 'numpy')


In [19]:
root, iterations = newton_raphson(f_num, f_prime_num, x0=1)

In [20]:
plot_newton_plotly(f_num,root, iterations)